<a href="https://colab.research.google.com/github/ben854719/Retention-Engagement-Assistant-Smart-Reminders-for-Customer-Success/blob/main/Agentic_AI_Avatar_Medical_Assistant_streamlit_app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade langchain-google-genai google-generativeai langchain langgraph langsmith

  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain-1.1.2-py3-none-any.whl.metadata (4.9 kB)
  Using cached langgraph-1.0.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_core-1.1.0-py3-none-any.whl.metadata (3.6 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cach

In [ ]:
import os
os.environ["LANGSMITH_API_KEY"] = "LangSmith"

In [ ]:
!pip install "mcp[cli]"
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("GeminiTools")

@mcp.tool()
def search(query: str) -> list:
    # Your search logic here
    return ["Result 1", "Result 2"]

In [ ]:
!cd mcp-server-demo
!ls

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory
app.py	sample_data


In [ ]:
!cd mcp-server-demo && uv add langchain-google-genai langgraph langsmith

/bin/bash: line 1: cd: mcp-server-demo: No such file or directory


In [ ]:
!pip install streamlit

In [ ]:
%%writefile app.py
import streamlit as st

Overwriting app.py


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_core.tools import Tool
from langgraph.graph import StateGraph, END
from IPython.display import display, HTML
from typing import TypedDict
import os
import random
import json
from mcp.server.fastmcp import FastMCP
from google.colab import userdata
from langsmith import traceable
from google.colab import userdata

# Import APl Key.
api_key = userdata.get("Ben856")
if not api_key:
    raise ValueError("Ben856 secret not found. Please set your API key in Colab Secrets.")
os.environ["GOOGLE_API_KEY"] = api_key

# Agent State Definition.
class AgentState(TypedDict):
   input: str
   output: str
   trace_id: str
   context: dict

# Prompt Template.
prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "I am your AI Assistant, here to help you stay on top of insurance payments, monitor your health status, track your medication intake, and provide timely alerts for any health concerns to keep you engaged and reassured."
    ),
    HumanMessagePromptTemplate.from_template("Respond confidently to: {input}")
])

# Initialize Gemini Model.
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key=api_key)
chain = prompt | llm

# Dynamic HeyGen Video Selector.
def select_heygen_url(response_text: str) -> str:
    if "onboarding" in response_text.lower():
        return "https://app.heygen.com/videos/your_onboarding_video_id"
    elif "risk" in response_text.lower() or "concern" in response_text.lower():
        return "https://app.heygen.com/videos/your_fallback_video_id"
    else:
        return "https://app.heygen.com/videos/705ca033b8614046b9efb8c0e07ccff2"

# LangGrap nodes.
@traceable(name="entry_node")
def entry_node(state: AgentState) -> AgentState:
    return {**state, "trace_id": f"trace-{random.randint(1000,9999)}"}

# Avatar Node (Colab-safe link preview).
def avatar_node(input_text):
    print("Avatar says:", input_text)
    heygen_url = select_heygen_url(input_text)
    display(HTML(f'<a href="{heygen_url}" target="_blank">▶️ Click to view avatar response</a>'))
    return {"avatar_output": f"Avatar says: {input_text}"}

# Optional Tool Wrapper.
avatar_tool = Tool.from_function(
    func=avatar_node,
    name="AvatarSpeaker",
    description="Trigger avatar speech"
)

# Workflow Nodes.
def gemini_node(state: "WorkflowState"):
    response = chain.invoke({"input": state["input"]})
    state["response"] = response.content
    return state

def emotion_node(state: "WorkflowState"):
    text = state["response"]
    if "confident" in text or "welcome" in text:
        state["emotion"] = "positive"
    elif "concern" in text or "risk" in text:
        state["emotion"] = "negative"
    else:
        state["emotion"] = "neutral"
    return state

def fallback_node(state: "WorkflowState"):
    if state["emotion"] == "Supportive":
        state["response"] = "Positive message: insurance payments, monitor your health status, track your medication intake, and provide timely alerts for any health concerns to keep you engaged and reassured."
    return state

def onboarding_node(state: "WorkflowState"):
    if "onboarding" in state["response"].lower():
        state["response"] = "Good day! I’m your Assistant, here to reassure you and engage with you every step of the way."
    return state

def compliance_score_node(state: "WorkflowState"):
    # Placeholder for compliance score logic
    state["compliance_score"] = random.randint(0, 100) # Example score
    return state

def avatar_speak_node(state: "WorkflowState"):
    avatar_result = avatar_node(state["response"])
    state["avatar_output"] = avatar_result["avatar_output"]
    state["input"] = state.get("input", "")
    return state

# State Schema
class WorkflowState(TypedDict):
    input: str
    response: str
    emotion: str
    avatar_output: str
    compliance_score: int # Add compliance_score to the state

# Langgraph flow.
graph = StateGraph(WorkflowState)
graph.add_node("gemini", gemini_node)
graph.add_node("emotion", emotion_node)
graph.add_node("fallback", fallback_node)
graph.add_node("onboarding", onboarding_node)
graph.add_node("compliance_score", compliance_score_node)
graph.add_node("avatar_speak", avatar_speak_node)

graph.set_entry_point("gemini")
graph.add_edge("gemini", "emotion")
graph.add_edge("emotion", "fallback")
graph.add_edge("fallback", "onboarding")
graph.add_edge("onboarding", "compliance_score")
graph.add_edge("compliance_score", "avatar_speak")
graph.add_edge("avatar_speak", END)

flow = graph.compile()

#  Run the Flow.
initial_state = {"input": "Generate a confident greeting for onboarding"}
final_state = flow.invoke(initial_state)

#  Print and Save Final State.
print("Final State:", final_state)

with open("avatar_response.json", "w") as f:
    json.dump(final_state, f, indent=2)

Avatar says: Good day! I’m your Assistant, here to reassure you and engage with you every step of the way.


Final State: {'input': 'Generate a confident greeting for onboarding', 'response': 'Good day! I’m your Assistant, here to reassure you and engage with you every step of the way.', 'emotion': 'positive', 'avatar_output': 'Avatar says: Good day! I’m your Assistant, here to reassure you and engage with you every step of the way.', 'compliance_score': 8}
